# Testing `models` module

**Authorship:**
Adam Klie, *03/19/2022*
***
**Description:**
Notebook for testing out EUGENE architectures

<div class="alert alert-block alert-warning">
<b>TODOs</b>:
<ul>
    <b><li></li></b>
    </ul>
</div>

# Set-up

In [1]:
import numpy as np
import pandas as pd
import torch

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

In [2]:
import eugene as eu

Global seed set to 13


# Load data

In [3]:
x = torch.randn(10, 4, 66)
x_t = x.transpose(2, 1)
x_rev = torch.randn(10, 4, 66)
x_rev_t = x_rev.transpose(2, 1)

# FCN Only

In [10]:
fcn = eu.models.FCN(input_len=66, 
          strand="ss",
          task="binary_classification",
          fc_kwargs=dict(hidden_dims=[100], output_dim=1))
fcn

/cellar/users/aklie/opt/miniconda3/envs/eugene/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


FCN(
  (fcn): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=264, out_features=100, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=100, out_features=1, bias=True)
    )
  )
  (accuracy): Accuracy()
  (auroc): AUROC()
)

In [11]:
out = fcn(x, x_rev)
out, out.shape

(tensor([[ 0.1548],
         [ 0.1647],
         [ 0.1098],
         [-0.0852],
         [ 0.3124],
         [ 0.0656],
         [-0.2696],
         [-0.1712],
         [-0.1495],
         [-0.1621]], grad_fn=<AddmmBackward0>),
 torch.Size([10, 1]))

# CNN Only

In [12]:
cnn = eu.models.CNN(input_len=66, 
          strand="ss",
          task="binary_classification", 
          conv_kwargs=dict(channels=[4, 16, 32], conv_kernels=[15, 5, 5], pool_kernels=[1, 1], pool_strides = [1, 1], dropout_rates=0.2), 
          fc_kwargs=dict(output_dim=1))
cnn

/cellar/users/aklie/opt/miniconda3/envs/eugene/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


CNN(
  (convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(15,), stride=(1,))
      (1): ReLU(inplace=True)
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.2, inplace=False)
      (4): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
      (5): ReLU(inplace=True)
      (6): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (7): Dropout(p=0.2, inplace=False)
    )
  )
  (fcnet): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=1536, out_features=1, bias=True)
    )
  )
  (accuracy): Accuracy()
  (auroc): AUROC()
)

In [14]:
out = cnn(x, x_rev)
out, out.shape

(tensor([[-0.0206],
         [ 0.0497],
         [ 0.0494],
         [-0.0139],
         [ 0.0993],
         [ 0.0783],
         [ 0.2109],
         [ 0.0920],
         [-0.0542],
         [ 0.0899]], grad_fn=<AddmmBackward0>),
 torch.Size([10, 1]))

# RNN Only

In [15]:
rnn = eu.models.RNN(input_len=66, 
          strand="ss",
          task="regression",
          rnn_kwargs=dict(output_dim=32, unit_type="lstm", bidirectional=True, batch_first=True), 
          fc_kwargs=dict(output_dim=1))
rnn

RNN(
  (rnn): BasicRecurrent(
    (module): LSTM(4, 32, batch_first=True, bidirectional=True)
  )
  (fcnet): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=64, out_features=1, bias=True)
    )
  )
  (r_squared): R2Score()
)

In [16]:
out = rnn(x_t, x_rev_t)
out, out.shape

(tensor([[ 0.0410],
         [ 0.0373],
         [-0.0069],
         [ 0.0746],
         [ 0.0769],
         [ 0.0542],
         [ 0.0185],
         [ 0.0386],
         [ 0.0927],
         [ 0.1275]], grad_fn=<AddmmBackward0>),
 torch.Size([10, 1]))

# Hybrid CNN-RNN

In [17]:
cnn_rnn = eu.models.Hybrid(input_len=66, 
                 strand="ts",
                 task="binary_classification", 
                 conv_kwargs=dict(channels=[4, 16, 32], conv_kernels=[15, 5], pool_kernels=[1, 1, 1], pool_strides = [1, 1], dropout_rates=0.2), 
                 rnn_kwargs=dict(output_dim=32, unit_type="lstm", bidirectional=True, batch_first=True),  
                 fc_kwargs=dict(output_dim=1))
cnn_rnn

/cellar/users/aklie/opt/miniconda3/envs/eugene/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


Hybrid(
  (convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(15,), stride=(1,))
      (1): ReLU(inplace=True)
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.2, inplace=False)
      (4): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
      (5): ReLU(inplace=True)
      (6): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (7): Dropout(p=0.2, inplace=False)
    )
  )
  (reverse_convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(15,), stride=(1,))
      (1): ReLU(inplace=True)
      (2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.2, inplace=False)
      (4): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
      (5): ReLU(inplace=True)
      (6): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
      (7): Dropout(p=0.2, inplace=False)
    )
  )
  (r

In [18]:
out = cnn_rnn(x, x_rev)
out, out.shape

(tensor([[-0.0302],
         [-0.0532],
         [-0.0707],
         [-0.0539],
         [-0.0465],
         [-0.0774],
         [-0.0164],
         [-0.0476],
         [-0.0719],
         [-0.0618]], grad_fn=<AddmmBackward0>),
 torch.Size([10, 1]))

# DeepBind

In [20]:
deepbind = eu.models.DeepBind(input_len=66)

In [21]:
# x_rev does not need to be specified in single strand mode
out = deepbind(x)

out, out.shape

(tensor([[-0.0146],
         [ 0.0010],
         [ 0.0380],
         [-0.0201],
         [ 0.0266],
         [-0.0139],
         [ 0.0040],
         [ 0.0325],
         [-0.0869],
         [ 0.0088]], grad_fn=<AddmmBackward0>),
 torch.Size([10, 1]))

# Load from config

In [47]:
model_config = "../_configs/test_fcn.yaml"
eu.models.load_config("FCN", model_config)

FCN(
  (fcn): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=264, out_features=128, bias=True)
      (1): ReLU(inplace=True)
      (2): Linear(in_features=128, out_features=64, bias=True)
      (3): ReLU(inplace=True)
      (4): Linear(in_features=64, out_features=1, bias=True)
    )
  )
  (r_squared): R2Score()
)

In [40]:
test_dict = {'input_len': 66, 'strand': 'ss', 'fc_kwargs': {'output_dim': 1, 'hidden_dims': [128, 66]}}

In [44]:
out = "../_configs/test_fcn.yaml"

In [42]:
test_dict

{'input_len': 66,
 'strand': 'ss',
 'fc_kwargs': {'output_dim': 1, 'hidden_dims': [128, 66]}}

In [45]:
import yaml
with open(out, 'w') as yaml_file:
    yaml.dump(test_dict, yaml_file, sort_keys=False, default_flow_style=False)

---

# Scratch